# Datafonos

Uno de nuestros clientes, **CLAP**, maneja Datafonos y se encarga de procesar
pagos. **SIMETRIK** es su aliado para comparar las transacciones que se
reportan a través de sus dispositivos contra las
transacciones que se registran en las liquidaciones
bancarias (**BANSUR**).  
Cada vez que el cliente **CLAP** reporta un pago con su
datáfono envía un registro a su base de datos y a la base
de datos del adquirente **BANSUR**. Sin embargo, dada la
gran cantidad de pagos que se realizan a diario, **CLAP** no puede llevar un
control efectivo de todas las transacciones realizadas y necesitan verificar
que todas las transacciones registradas en su base de datos también se
encuentren en la base de **BANSUR**.

*IMPORTANTE:* Una transacción regular se evidencia en la base de datos
como un **PAGO**; se debe tener en cuenta que un *mismo ID* puede también
tomar estado de **Cancelación**, **Chargeback** u **Otros casos**.  
Simetrik considera una partida como conciliable toda aquella transacción
cuyo último estado en la base de datos ordenada por fecha y hora sea
PAGADA.  
Para esto, **SIMETRIK** comparará para cada transacción campos únicos entre
las dos entidades buscando encontrar parejas que sean exactamente
iguales bajo las siguientes condiciones:  
1. Que tengan el mismo **ID**.
2. Que tenga los mismos 6 primeros dígitos de la tarjeta.
3. Que tengan los mismos 4 últimos dígitos de la tarjeta.
4. Que el valor pagado en la transacción sea igual o que su diferencia
esté en el rango de más o menos 0.99 pesos.
5. Que tengan la misma fecha de transacción.

###  Importo las bibliotecas necesarias:

In [329]:
import pandas as pd
from sqlalchemy import create_engine, Column, Integer, String, Float, DateTime
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

### Cargo los datos de los archivos CSV en DataFrames de Pandas:

In [318]:
clap_df = pd.read_csv('data/clap.csv')
bansur_df = pd.read_csv('data/bansur.csv')

In [319]:
print(clap_df.dtypes)

INICIO6_TARJETA          float64
FINAL4_TARJETA             int64
TIPO_TRX                  object
MONTO                    float64
FECHA_TRANSACCION         object
CODIGO_AUTORIZACION       object
ID_BANCO                 float64
FECHA_RECEPCION_BANCO     object
dtype: object


In [320]:
print(bansur_df.dtypes)

TARJETA                  int64
TIPO_TRX                object
MONTO                  float64
FECHA_TRANSACCION        int64
CODIGO_AUTORIZACION     object
ID_ADQUIRIENTE           int64
FECHA_RECEPCION         object
dtype: object


In [321]:
# Revisando datos faltantes
print(clap_df.isna().sum())
print(bansur_df.isna().sum())

INICIO6_TARJETA              6
FINAL4_TARJETA               0
TIPO_TRX                     0
MONTO                        0
FECHA_TRANSACCION            0
CODIGO_AUTORIZACION      12679
ID_BANCO                    16
FECHA_RECEPCION_BANCO        0
dtype: int64
TARJETA                0
TIPO_TRX               0
MONTO                  0
FECHA_TRANSACCION      0
CODIGO_AUTORIZACION    0
ID_ADQUIRIENTE         0
FECHA_RECEPCION        0
dtype: int64


In [322]:
# Rellenar columnas 
clap_df['CODIGO_AUTORIZACION'] = clap_df['CODIGO_AUTORIZACION'].fillna(0)
clap_df['INICIO6_TARJETA'] = clap_df['INICIO6_TARJETA'].fillna(0)
clap_df['ID_BANCO'] = clap_df['ID_BANCO'].fillna(0)

In [326]:
# Convertir 'ID_BANCO' a int64
clap_df['ID_BANCO'] = clap_df['ID_BANCO'].astype('int64') 

In [323]:
# Asegurarme que las fechas esten en el mismo formato

# Convertir la columna 'FECHA_TRANSACCION' a objetos datetime.date
bansur_df['FECHA_TRANSACCION'] = bansur_df['FECHA_TRANSACCION'].astype(str)
bansur_df['FECHA_TRANSACCION'] = pd.to_datetime(bansur_df['FECHA_TRANSACCION'])
bansur_df['FECHA_TRANSACCION'] = bansur_df['FECHA_TRANSACCION'].dt.date

# Convertir la columna 'FECHA_RECEPCION' a objetos datetime.date
bansur_df['FECHA_RECEPCION'] = pd.to_datetime(bansur_df['FECHA_RECEPCION']).dt.date

# Convertir la columna 'FECHA_RECEPCION_BANCO' a objetos datetime.date
clap_df['FECHA_RECEPCION_BANCO'] = pd.to_datetime(clap_df['FECHA_RECEPCION_BANCO']).dt.date

# Convertir la columna 'FECHA_TRANSACCION' a objetos datetime.date
clap_df['FECHA_TRANSACCION'] = pd.to_datetime(clap_df['FECHA_TRANSACCION']).dt.date


In [324]:
# Ordeno los registros para poder trabajar en el orden cronológico correcto
clap_df = clap_df.sort_values(by='FECHA_TRANSACCION')
bansur_df = bansur_df.sort_values(by='FECHA_TRANSACCION')

In [327]:
print(clap_df.head())

        INICIO6_TARJETA  FINAL4_TARJETA   TIPO_TRX    MONTO FECHA_TRANSACCION  \
0              443715.0             190  CANCELADA  21944.0        2020-11-01   
109028         442753.0            2659     PAGADA     31.0        2020-11-01   
109029         905158.0             278     PAGADA     35.0        2020-11-01   
109030         998516.0            6919     PAGADA    231.0        2020-11-01   
109031         438907.0            8349     PAGADA     90.0        2020-11-01   

       CODIGO_AUTORIZACION      ID_BANCO FECHA_RECEPCION_BANCO  
0                   136719   47100185686            2020-11-05  
109028              060126  160306336072            2020-11-02  
109029              096329  832092699244            2020-11-02  
109030              080694  839451917579            2020-11-02  
109031              697577  862853928650            2020-11-02  


In [328]:
print(bansur_df.head())

          TARJETA TIPO_TRX  MONTO FECHA_TRANSACCION CODIGO_AUTORIZACION  \
0      9540150300     PAGO   44.0        2020-11-01              272012   
88272  9540159834     PAGO  384.0        2020-11-01              235583   
88271  9540159834     PAGO  304.0        2020-11-01              987379   
88270  9540159831     PAGO  451.0        2020-11-01              095554   
88269  9540159830     PAGO  190.0        2020-11-01              866583   

       ID_ADQUIRIENTE FECHA_RECEPCION  
0        320720237226      2020-11-03  
88272    828295617069      2020-11-03  
88271    335823176959      2020-11-03  
88270    873463833342      2020-11-03  
88269    188703426158      2020-11-03  


###  Defino la conexión a la base de datos y creamos una sesión:

In [331]:
# Parámetros de la conexión a la base de datos
db_params = {
    'user': 'postgres',
    'password': 'Jmqr3st0',
    'host': 'localhost',
    'port': '5432',
    'database': 'db'
}

conn_str = "postgresql://{user}:{password}@{host}:{port}/{database}".format(**db_params)
engine = create_engine(conn_str)

# Se crea una sesión
Session = sessionmaker(bind=engine)
session = Session()

###  Defino la estructura de las tablas utilizando SQLAlchemy ORM:

In [333]:

Base = declarative_base()

class Clap(Base):
    __tablename__ = 'clap'

    id = Column(Integer, primary_key=True)
    inicio6_tarjeta = Column(Integer)
    final4_tarjeta = Column(Integer)
    tipo_trx = Column(String)
    monto = Column(Float)
    fecha_transaccion = Column(DateTime)
    codigo_autorizacion = Column(String)
    id_banco = Column(Integer)
    fecha_recepcion_banco = Column(DateTime)

class Bansur(Base):
    __tablename__ = 'bansur'

    id = Column(Integer, primary_key=True)
    tarjeta = Column(String)
    tipo_trx = Column(String)
    monto = Column(Float)
    fecha_transaccion = Column(DateTime)
    codigo_autorizacion = Column(String)
    id_adquiriente = Column(Integer)
    fecha_recepcion = Column(DateTime)


C:\Users\CtrlCri\AppData\Local\Temp\ipykernel_9272\3609720198.py:1: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


### Creo las tablas en la base de datos:

In [334]:
Base.metadata.create_all(engine)

### Retomando las condiciones:
1. Que tengan el mismo **ID**.
2. Que tenga los mismos 6 primeros dígitos de la tarjeta.
3. Que tengan los mismos 4 últimos dígitos de la tarjeta.
4. Que el valor pagado en la transacción sea igual o que su diferencia
esté en el rango de más o menos 0.99 pesos.
5. Que tengan la misma fecha de transacción.